In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, InputLayer, Input, add, dot, Dot, maximum, average, multiply, concatenate, Reshape

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow import linalg #LinearOperatorCirculant
import numpy as np
import pandas as pd
from scipy.linalg import circulant

In [3]:
# reading data
data = pd.read_csv('PrOCTOR_sample_data_all.csv', header=0)
data1 = data.fillna(data.mean()['MolecularWeight':'Salivary Gland'])
data1["target"] = np.where(data1.iloc[:, 1] == "passed", 1, 0)

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# x, y variable
X = data1.iloc[:, 2:-1]
y = data1['target']
#print(X)

In [5]:
X = np.array(X)
X = scaler.fit_transform(X)
y = np.array(y)
y_1 = y.reshape(y.shape[0],-1)
y_predict = y_1
#y = scaler.fit_transform(X)
des=X[:,:13]
body=X[:,13:]

_, des_col = des.shape
_, body_col = body.shape

In [6]:
print(des.shape)
print(body.shape)
print(y.shape)
print(y_1.shape)

(828, 13)
(828, 34)
(828,)
(828, 1)


In [7]:
hid_layer = [100, 100, 50]
output_layer = [2601]
print(hid_layer[-1])

50


In [8]:
#from keras.layers import Merge

class MM_Sum:
    def __init__(self, des_shape, body_shape, hid_layer, output_layer):
        self.des_shape = des_shape
        self.body_shape = body_shape
        self.hid_layer = hid_layer  # not used
        self.output_layer = output_layer
        
    def Multi_modal_Sum(self):
        # bias modality
        bias_input = Input(shape = (1,))
        
        # des modality
        des_input = Input(shape = self.des_shape) # input = 17, des_shape = 17
        des_input_x1 = Dense(self.hid_layer[0], activation = 'relu')(des_input)#(des_input_flat)
        des_input_x1_batch = BatchNormalization()(des_input_x1)
        des_input_x3 = Dense(self.hid_layer[2], activation = 'relu')(des_input_x1_batch)
        des_input_x3_batch = BatchNormalization()(des_input_x3)
        
        # body modality
        body_input = Input(shape = self.body_shape) # input = 30,body_shape = 30
        body_input_x1 = Dense(self.hid_layer[0], activation = 'relu')(body_input)#(des_input_flat)
        body_input_x1_batch = BatchNormalization()(body_input_x1)
        body_input_x3 = Dense(self.hid_layer[2], activation = 'relu')(body_input_x1_batch)
        body_input_x3_batch = BatchNormalization()(body_input_x3)
        
        concat_layer1 = concatenate([bias_input, des_input_x3_batch])
        concat_layer2 = concatenate([bias_input, body_input_x3_batch])
        
        re_concat1 = Reshape((1, 51))(concat_layer1)
        re_concat2 = Reshape((1, 51))(concat_layer2)
        
        dot_layer = dot([re_concat1, re_concat2], axes = 1, normalize=False)
        tensor_flat = Flatten()(dot_layer)
        fusion_dense = Dense(self.hid_layer[2], activation = 'relu')(tensor_flat)
        fusion_out = Dense(self.output_layer, activation = 'sigmoid')(fusion_dense)
        
        fusion_model = keras.models.Model(inputs = [bias_input, des_input, body_input], outputs = [fusion_out])
        
        return fusion_model

In [9]:
final_model = MM_Sum(des_col, body_col, hid_layer, 1) # input type = integer
network = final_model.Multi_modal_Sum()
print(network)

In [10]:
network.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 100)          1400        input_2[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 100)          3500        input_3[0][0]                    
______________________________________________________________________________________________

In [11]:
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

# 모델설정
sm = SMOTE(random_state=202004)
loo = LeaveOneOut()
kfold = KFold(n_splits = 10, shuffle = True, random_state = 111)
kfold.get_n_splits(X)

10

In [12]:
def predict_change(predict):
    y_predict = []
    for i in range(len(predict)):
        y_predict = np.append(y_predict, predict[i])
    
    y_predict = np.array(y_predict)
    y_predict = y_predict.reshape(y_predict.shape[0],-1)
    return y_predict

In [13]:
neg, pos = np.bincount(y)
total = neg + pos

weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 5.83
Weight for class 1: 0.55


In [14]:
dict_y = dict()

class train_model:
    def __init__(self, x, y, model, pd_y, pd_sm_y, pd_cw_y, pd_smcw_y):
        self.X = x
        self.y = y
        self.model = model
        self.pd_y = pd_y
        self.pd_sm_y = pd_sm_y
        self.pd_cw_y = pd_cw_y
        self.pd_smcw_y = pd_smcw_y
        
    def train_based(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [np.ones(des_train.shape[0]),des_train, body_train], y = y_train, epochs=50)

            y_pred = self.model.predict([np.ones(des_test.shape[0]), des_test[:], body_test[:]])

            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
                
        return self.pd_y
    
    def train_smote(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)

            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [np.ones(sm_des_train.shape[0]), sm_des_train, sm_body_train], y = sm_y_train, epochs=50)

            y_pred = self.model.predict([np.ones(des_test.shape[0]), des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_sm_y
    
    def train_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
            
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [np.ones(des_train.shape[0]),des_train, body_train], y = y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([np.ones(des_test.shape[0]), des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_cw_y
    
    def train_sm_weight(self):
        for train_index, test_index in kfold.split(self.X):
            print("TEST:", test_index)
            des_train, des_test = des[train_index], des[test_index]
            body_train, body_test = body[train_index], body[test_index]
            y_train, y_test = y[train_index], y[test_index]

            neg, pos = np.bincount(y_train)
            total = neg + pos

            weight_for_0 = (1 / neg)*(total)/2.0 
            weight_for_1 = (1 / pos)*(total)/2.0

            class_weight = {0: weight_for_0, 1: weight_for_1}
    
            sm_des_train, sm_y_train = sm.fit_sample(des_train, y_train)
            sm_body_train, _ = sm.fit_sample(body_train, y_train)
    
            self.model.compile(optimizer= Adam(), loss='binary_crossentropy', metrics = ['accuracy'])
            kf_history = self.model.fit(x = [np.ones(sm_des_train.shape[0]), sm_des_train, sm_body_train], y = sm_y_train, epochs=50, class_weight = class_weight)

            y_pred = self.model.predict([np.ones(des_test.shape[0]), des_test[:], body_test[:]])
            
            for i in range(len(test_index)):
                index = test_index[i]
                self.pd_y[index] = y_pred[i]
            
        return self.pd_smcw_y
    

In [15]:
final_model = MM_Sum(des_col, body_col, hid_layer, 1) # input type = integer
network = final_model.Multi_modal_Sum()
print(network)

f_y, f_sm_y, f_cw_y,f_smcw_y = [], [], [], []
output_fusion = train_model(X, y, network, dict_y, dict_y, dict_y, dict_y)

# sum model training
f_y = output_fusion.train_based()
#f_sm_y = output_fusion.train_smote()
#f_cw_y = output_fusion.train_weight()
#f_smcw_y = output_fusion.train_sm_weight()

TEST: [  1  11  12  34  36  49  56  71  93 112 119 125 126 140 145 152 157 183
 190 194 198 205 209 210 211 214 233 237 257 264 272 298 303 323 328 340
 354 368 373 380 399 403 409 412 422 432 434 447 456 461 464 468 474 476
 491 496 506 518 520 523 528 532 538 547 553 556 589 611 624 629 645 658
 665 700 702 708 735 757 766 774 799 800 825]
Epoch 1/50
24/24 [==============================] - 0s 3ms/step - loss: 0.9332 - accuracy: 0.5490
Epoch 2/50
24/24 [==============================] - 0s 4ms/step - loss: 0.2703 - accuracy: 0.9114
Epoch 3/50
24/24 [==============================] - 0s 4ms/step - loss: 0.1512 - accuracy: 0.9530
Epoch 4/50
24/24 [==============================] - 0s 4ms/step - loss: 0.1243 - accuracy: 0.9530
Epoch 5/50
24/24 [==============================] - 0s 4ms/step - loss: 0.1200 - accuracy: 0.9664
Epoch 6/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0960 - accuracy: 0.9758
Epoch 7/50
24/24 [==============================] - 0s 3ms/step - lo

24/24 [==============================] - 0s 3ms/step - loss: 0.0207 - accuracy: 0.9919
Epoch 2/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0257 - accuracy: 0.9919
Epoch 3/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0081 - accuracy: 0.9960
Epoch 4/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0101 - accuracy: 0.9946
Epoch 5/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0176 - accuracy: 0.9946
Epoch 6/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0249 - accuracy: 0.9933
Epoch 7/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0363 - accuracy: 0.9946
Epoch 8/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0258 - accuracy: 0.9933
Epoch 9/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0134 - accuracy: 0.9946
Epoch 10/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0123 - accuracy: 0.9946
Epoch 11/50
24/24 [===========

24/24 [==============================] - 0s 4ms/step - loss: 0.0076 - accuracy: 0.9973
Epoch 6/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0094 - accuracy: 0.9960
Epoch 7/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 8/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 9/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0042 - accuracy: 0.9973
Epoch 10/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 11/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0080 - accuracy: 0.9973
Epoch 12/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0162 - accuracy: 0.9987
Epoch 13/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0084 - accuracy: 0.9973
Epoch 14/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0138 - accuracy: 0.9987
Epoch 15/50
24/24 [=======

24/24 [==============================] - 0s 3ms/step - loss: 0.0045 - accuracy: 0.9987
Epoch 35/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0079 - accuracy: 0.9973
Epoch 36/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0237 - accuracy: 0.9933
Epoch 37/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0272 - accuracy: 0.9919
Epoch 38/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0042 - accuracy: 0.9987
Epoch 39/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0280 - accuracy: 0.9946
Epoch 40/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0175 - accuracy: 0.9919
Epoch 41/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0072 - accuracy: 0.9960
Epoch 42/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0127 - accuracy: 0.9973
Epoch 43/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0136 - accuracy: 0.9933
Epoch 44/50
24/24 [===

In [16]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc, f1_score, confusion_matrix, accuracy_score, matthews_corrcoef, accuracy_score
from imblearn.metrics import geometric_mean_score
import math

class Evaluation:
    def __init__(self, pred, y):
        self.y_pred = pred
        self.y = y
        
    def matrix(self):
        y_1 = self.y.reshape(self.y.shape[0],-1)
        y_pred = np.array(self.y_pred)
        y_pred = y_pred.reshape(y_pred.shape[0], -1)
        
        y_classify = []
        for i in range(len(self.y_pred)):
            if self.y_pred[i] >= 0.5:
                a = 1.
                y_classify.append(a)
            else:
                a = 0.
                y_classify.append(a)
        
        fpr,tpr,threshold = roc_curve(y_1 , y_pred, pos_label = 1)
        precision, recall, threshold = precision_recall_curve(y_1, y_pred, pos_label = 1)
        
        roc_auc = auc(fpr,tpr)
        auprc = auc(recall, precision)
        mean_precision = np.mean(precision)
        mean_recall = np.mean(recall)
        F1 = 2 * (mean_precision * mean_recall) / (mean_precision + mean_recall)
        # binary 
        accuracy = accuracy_score(y_1, y_classify)
        mcc = matthews_corrcoef(y_1, y_classify) 
        g_mean = geometric_mean_score(y_1, y_classify)
        confusion = confusion_matrix(y_1, y_classify)
        print(confusion.ravel())
        tn, fp, fn, tp = confusion.ravel()
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)
        ppv = tp / (tp + fp)
        fnr = fn / (fn + tp)
        fpr = fp / (fp + tn)

        confu_precision = ppv 
        confu_recall = tpr # sensitivity
        confu_f1 = 2 * ((ppv * tpr) / (ppv + tpr))
        confu_accuracy = (tp + tn) / (tp + tn + fp + fn)
        confu_mcc = ((tp * tn)-(fp-fn))/ math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))
        confu_g_mean = math.sqrt(tpr * tnr)
        Optimized_precision = (confu_accuracy - abs(tnr-tpr)) / (tnr + tpr)

        print('공통 \nAUC :',roc_auc) # pb
        print("AUPRC :", auprc) # pb
        print("Optimized precision :", Optimized_precision)

        print("\nfunction 사용\nAccuracy :", accuracy) #pb
        print("Precision(pb) :",mean_precision )
        print("Recall(pb) :", mean_recall) # pb
        print("F1 score(pb) :", F1) #pb

        print("MCC :", mcc)
        print("G-mean :", g_mean)


        print("\nConfusion_matrix 사용 \n", confusion)
        print("Accuracy :", confu_accuracy)
        print("Precision :", confu_precision) 
        print("Recall :", confu_recall) 
        print("F1 score :", confu_f1) 

        print("MCC :", confu_mcc)
        print("G-mean :", confu_g_mean)
        
        return roc_auc, auprc, Optimized_precision, accuracy, mean_precision, mean_recall, F1, mcc, g_mean, confu_accuracy, confu_precision, confu_recall, confu_f1, confu_mcc, confu_g_mean;

In [18]:
f_y_result = Evaluation(predict_change(f_y), y_1)
#f_sm_y_result = Evaluation(f_sm_y, y)
#f_cw_y_result = Evaluation(predict_change(f_cw_y), y)
#f_smcw_y_result = Evaluation(f_smcw_y, y)

print("#Fusion model 결과 \n")
print("\n-----based model-----\n")
print(f_y_result.matrix())
#
print("\n-----smote model-----\n")
#print(f_sm_y_result.matrix())

print("\n-----class weight model-----\n")
#print(f_cw_y_result.matrix())

print("\n-----smote + class weight model-----\n")
#print(f_smcw_y_result.matrix())

#Fusion model 결과 


-----based model-----

[ 52  19  20 737]
공통 
AUC : 0.9591791169739707
AUPRC : 0.9954697129511084
Optimized precision : 0.41718858345745063

function 사용
Accuracy : 0.9528985507246377
Precision(pb) : 0.9866380565169505
Recall(pb) : 0.8107018934390137
F1 score(pb) : 0.890059045954971
MCC : 0.7015180527454289
G-mean : 0.8444195929705219

Confusion_matrix 사용 
 [[ 52  19]
 [ 20 737]]
Accuracy : 0.9528985507246377
Precision : 0.9748677248677249
Recall : 0.9735799207397622
F1 score : 0.9742233972240582
MCC : 0.7085620749385558
G-mean : 0.8444195929705219
(0.9591791169739707, 0.9954697129511084, 0.41718858345745063, 0.9528985507246377, 0.9866380565169505, 0.8107018934390137, 0.890059045954971, 0.7015180527454289, 0.8444195929705219, 0.9528985507246377, 0.9748677248677249, 0.9735799207397622, 0.9742233972240582, 0.7085620749385558, 0.8444195929705219)

-----smote model-----


-----class weight model-----


-----smote + class weight model-----



c:\users\user\anaconda3\envs\gpu\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [144]:
fusion_y_set = []

for i in range(20):
    final_model = MM_Sum(des_col, body_col, hid_layer, 1) # input type = integer
    network = final_model.Multi_modal_Sum()
    print(network)

    f_y, f_sm_y, f_cw_y,f_smcw_y = [], [], [], []
    output_fusion = train_model(X, y, network, dict_y, dict_y, dict_y, dict_y)

    # sum model training
    f_y = output_fusion.train_based()
    #f_sm_y = output_fusion.train_smote()
    #f_cw_y = output_fusion.train_weight()
    #f_smcw_y = output_fusion.train_sm_weight()

    
    f_y_result = Evaluation(predict_change(f_y), y_1)
    #f_sm_y_result = Evaluation(f_sm_y, y)
    #f_cw_y_result = Evaluation(predict_change(f_cw_y), y)
    #f_smcw_y_result = Evaluation(predict_change(f_smcw_y), y)
    
    fusion_y_set = np.append(fusion_y_set, f_y_result.matrix())
    print(len(fusion_y_set))
    
fusion_y_np = np.array(fusion_y_set)

fusion_y_np = fusion_y_np.reshape(-1,15)
fusion_y_np.shape

TEST: [  1  11  12  34  36  49  56  71  93 112 119 125 126 140 145 152 157 183
 190 194 198 205 209 210 211 214 233 237 257 264 272 298 303 323 328 340
 354 368 373 380 399 403 409 412 422 432 434 447 456 461 464 468 474 476
 491 496 506 518 520 523 528 532 538 547 553 556 589 611 624 629 645 658
 665 700 702 708 735 757 766 774 799 800 825]
Epoch 1/50
24/24 [==============================] - 0s 3ms/step - loss: 0.4500 - accuracy: 0.8443
Epoch 2/50
24/24 [==============================] - 0s 3ms/step - loss: 0.2022 - accuracy: 0.9396
Epoch 3/50
24/24 [==============================] - 0s 3ms/step - loss: 0.1453 - accuracy: 0.9544
Epoch 4/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0965 - accuracy: 0.9678
Epoch 5/50
24/24 [==============================] - 0s 3ms/step - loss: 0.1232 - accuracy: 0.9638
Epoch 6/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0719 - accuracy: 0.9812
Epoch 7/50
24/24 [==============================] - 0s 3ms/step - lo

24/24 [==============================] - 0s 2ms/step - loss: 0.0038 - accuracy: 0.9987
Epoch 3/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0072 - accuracy: 0.9973
Epoch 4/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0089 - accuracy: 0.9960
Epoch 5/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0275 - accuracy: 0.9893
Epoch 6/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0825 - accuracy: 0.9866
Epoch 7/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0654 - accuracy: 0.9826
Epoch 8/50
24/24 [==============================] - ETA: 0s - loss: 0.0290 - accuracy: 0.9901   - 0s 2ms/step - loss: 0.0274 - accuracy: 0.9906
Epoch 9/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0113 - accuracy: 0.9973
Epoch 10/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0181 - accuracy: 0.9946
Epoch 11/50
24/24 [==============================] - 0s 2ms/step - loss: 0.0353 - 

24/24 [==============================] - 0s 3ms/step - loss: 0.0063 - accuracy: 0.9987
Epoch 32/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0089 - accuracy: 0.9960
Epoch 33/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 34/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0104 - accuracy: 0.9960
Epoch 35/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0090 - accuracy: 0.9973
Epoch 36/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 37/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0083 - accuracy: 0.9960
Epoch 38/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0219 - accuracy: 0.9919
Epoch 39/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0086 - accuracy: 0.9987
Epoch 40/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0095 - accuracy: 0.9946
Epoch 41/50
24/24 [===

Epoch 7/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0058 - accuracy: 0.9987
Epoch 8/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0040 - accuracy: 0.9987
Epoch 9/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0119 - accuracy: 0.9973
Epoch 10/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 11/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0058 - accuracy: 0.9973
Epoch 12/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0052 - accuracy: 0.9973
Epoch 13/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0047 - accuracy: 0.9973
Epoch 14/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0272 - accuracy: 0.9946
Epoch 15/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0031 - accuracy: 0.9987
Epoch 16/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 17/50
2

24/24 [==============================] - 0s 3ms/step - loss: 0.0053 - accuracy: 0.9973
Epoch 12/50
24/24 [==============================] - 0s 3ms/step - loss: 5.2913e-04 - accuracy: 1.0000
Epoch 13/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0081 - accuracy: 0.9973
Epoch 14/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0068 - accuracy: 0.9973
Epoch 15/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0168 - accuracy: 0.9987
Epoch 16/50
24/24 [==============================] - 0s 3ms/step - loss: 6.4056e-04 - accuracy: 1.0000
Epoch 17/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0038 - accuracy: 0.9987
Epoch 18/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0017 - accuracy: 0.9987
Epoch 19/50
24/24 [==============================] - 0s 3ms/step - loss: 6.6790e-04 - accuracy: 1.0000
Epoch 20/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0096 - accuracy: 0.9987
Epoch 21/5

...

24/24 [==============================] - 0s 3ms/step - loss: 0.0080 - accuracy: 0.9987
Epoch 26/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0022 - accuracy: 0.9987
Epoch 27/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0032 - accuracy: 0.9973
Epoch 28/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0034 - accuracy: 0.9987
Epoch 29/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 30/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0039 - accuracy: 0.9987
Epoch 31/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0042 - accuracy: 0.9973
Epoch 32/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0054 - accuracy: 0.9960
Epoch 33/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0071 - accuracy: 0.9987
Epoch 34/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 35/50
24/24 [===

24/24 [==============================] - 0s 3ms/step - loss: 0.0041 - accuracy: 0.9987
Epoch 24/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0092 - accuracy: 0.9973
Epoch 25/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0074 - accuracy: 0.9960
Epoch 26/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0127 - accuracy: 0.9960
Epoch 27/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0146 - accuracy: 0.9933
Epoch 28/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0159 - accuracy: 0.9973
Epoch 29/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0111 - accuracy: 0.9973
Epoch 30/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0104 - accuracy: 0.9946
Epoch 31/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0049 - accuracy: 0.9987
Epoch 32/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0137 - accuracy: 0.9946
Epoch 33/50
24/24 [===

Epoch 1/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0386 - accuracy: 0.9879
Epoch 2/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0246 - accuracy: 0.9919
Epoch 3/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0177 - accuracy: 0.9973
Epoch 4/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0190 - accuracy: 0.9933
Epoch 5/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0094 - accuracy: 0.9946
Epoch 6/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0064 - accuracy: 0.9973
Epoch 7/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0072 - accuracy: 0.9960
Epoch 8/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0090 - accuracy: 0.9946
Epoch 9/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0137 - accuracy: 0.9973
Epoch 10/50
24/24 [==============================] - 0s 4ms/step - loss: 0.0079 - accuracy: 0.9960
Epoch 11/50
24/24 [

24/24 [==============================] - 0s 2ms/step - loss: 0.0160 - accuracy: 0.9960
Epoch 6/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0063 - accuracy: 0.9973
Epoch 7/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0112 - accuracy: 0.9960
Epoch 8/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0154 - accuracy: 0.9960
Epoch 9/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0099 - accuracy: 0.9960
Epoch 10/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0062 - accuracy: 0.9973
Epoch 11/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0130 - accuracy: 0.9960
Epoch 12/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 13/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0147 - accuracy: 0.9973
Epoch 14/50
24/24 [==============================] - 0s 3ms/step - loss: 0.0299 - accuracy: 0.9933
Epoch 15/50
24/24 [=======

(20, 15)

In [79]:
def result(data):
    auc = data[:,0]
    auprc = data[:,1]
    op = data[:,2]
    acc = data[:,3]
    precision = data[:,4]
    recall = data[:,5]
    f1 = data[:,6]
    mcc = data[:,7]
    g_mean = data[:,8]
    confu_acc = data[:,9]
    confu_precision = data[:,10]
    confu_recall = data[:,11]
    confu_f1 = data[:,12]
    confu_mcc = data[:,13]
    confu_g_mean = data[:,14]

    print('공통 \nAUC :\n',"[평균 :",format(auc.mean(), ".5f"), "] [표준편차 :",format(auc.std(), ".5f"),"]") # pb
    print("AUPRC :\n", "[평균 :",format(auprc.mean(), ".5f"), "] [표준편차 :",format(auprc.std(), ".5f"),"]")
    print("Optimized precision :\n","[평균 :",format(op.mean(), ".5f"), "] [표준편차 :",format(op.std(), ".5f"),"]")

    print("\nfunction 사용\nAccuracy :\n","[평균 :",format(acc.mean(), ".5f"), "] [표준편차 :",format(acc.std(), ".5f"),"]")
    print("Precision(pb) :\n", "[평균 :",format(precision.mean(), ".5f"), "] [표준편차 :",format(precision.std(), ".5f"),"]")
    print("Recall(pb) :\n", "[평균 :",format(recall.mean(), ".5f"), "] [표준편차 :",format(recall.std(), ".5f"),"]")
    print("F1 score(pb) :\n","[평균 :",format(f1.mean(), ".5f"), "] [표준편차 :",format(f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(mcc.mean(), ".5f"), "] [표준편차 :",format(mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(g_mean.mean(), ".5f"), "] [표준편차 :",format(g_mean.std(), ".5f"),"]")
    
    print("\nConfusion matrix 사용 \nAccuracy :\n", "[평균 :",format(confu_acc.mean(), ".5f"), "] [표준편차 :",format(confu_acc.std(), ".5f"),"]")
    print("Precision :\n", "[평균 :",format(confu_precision.mean(), ".5f"), "] [표준편차 :",format(confu_precision.std(), ".5f"),"]")
    print("Recall :\n", "[평균 :",format(confu_recall.mean(), ".5f"), "] [표준편차 :",format(confu_recall.std(), ".5f"),"]")
    print("F1 score :\n", "[평균 :",format(confu_f1.mean(), ".5f"), "] [표준편차 :",format(confu_f1.std(), ".5f"),"]")
    print("MCC :\n", "[평균 :",format(confu_mcc.mean(), ".5f"), "] [표준편차 :",format(confu_mcc.std(), ".5f"),"]")
    print("G-mean :\n", "[평균 :",format(confu_g_mean.mean(), ".5f"), "] [표준편차 :",format(confu_g_mean.std(), ".5f"),"]")
    

In [145]:
result(fusion_y_np)

공통 
AUC :
 [평균 : 0.96799 ] [표준편차 : 0.00712 ]
AUPRC :
 [평균 : 0.99634 ] [표준편차 : 0.00096 ]
Optimized precision :
 [평균 : 0.42329 ] [표준편차 : 0.01331 ]

function 사용
Accuracy :
 [평균 : 0.96389 ] [표준편차 : 0.00526 ]
Precision(pb) :
 [평균 : 0.98874 ] [표준편차 : 0.00253 ]
Recall(pb) :
 [평균 : 0.84231 ] [표준편차 : 0.01267 ]
F1 score(pb) :
 [평균 : 0.90961 ] [표준편차 : 0.00698 ]
MCC :
 [평균 : 0.76329 ] [표준편차 : 0.03420 ]
G-mean :
 [평균 : 0.86218 ] [표준편차 : 0.02096 ]

Confusion matrix 사용 
Accuracy :
 [평균 : 0.96389 ] [표준편차 : 0.00526 ]
Precision :
 [평균 : 0.97729 ] [표준편차 : 0.00325 ]
Recall :
 [평균 : 0.98336 ] [표준편차 : 0.00365 ]
F1 score :
 [평균 : 0.98031 ] [표준편차 : 0.00288 ]
MCC :
 [평균 : 0.76743 ] [표준편차 : 0.03297 ]
G-mean :
 [평균 : 0.86218 ] [표준편차 : 0.02096 ]
